## Importar librerias necesarias, trabajomos con nasdaq data link

In [2]:
import nasdaqdatalink as ndl
import pandas as pd
from ta import add_all_ta_features
import pandas as pd
import zipfile
import json

## (Opcional) Visualizar máximo número de filas y columnas en df

In [2]:
#configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Configuración de API key

In [3]:

with open(r'..\config\ndl-config.json') as f:
    config = json.load(f)

api_key = config['API_KEY']
ndl.ApiConfig.api_key = api_key


## Definimos el inicio de nuestro dataset

In [4]:
start_date = '1998-01-01'

## Descarga Constituyentes Históricos SP500

In [5]:
all = ndl.get_table("SHARADAR/SP500", action = "historical", date={'gte':start_date}, paginate=True)

# filtrar solo para final del Q1 cada año, (Marzo) esta será nuestra fecha de rabalanceo del portfolio
sp500 = all[all['date'].dt.month == 3]
sp500.drop(columns=['action', 'name', 'contraticker', 'contraname','note'], inplace=True)
sp500.head()

C:\Users\Ignacio\AppData\Local\Temp\ipykernel_14756\1897431454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500.drop(columns=['action', 'name', 'contraticker', 'contraname','note'], inplace=True)


,date,ticker
None,,
0,2025-03-31,ZTS
1,2025-03-31,ZBRA
2,2025-03-31,ZBH
3,2025-03-31,YUM
4,2025-03-31,XYL


In [6]:
sp500.to_csv("../data/sp500.csv", index=False)

#### Número de tickers participantes históricos en el sp500

In [7]:
sp500['ticker'].nunique()

1132

#### Verificamos que tenemos aproximadamente 500 tickers para cada año

In [8]:
sp500[sp500['date'] =="2010-03-31"]['ticker'].nunique()

500

## 2. Descarga fundamentales trimestrales (ARY) desde 1998


In [19]:
tickers = sp500['ticker'].tolist()

In [ ]:
fundamental = ndl.get_table(
    "SHARADAR/SF1",
    ticker=tickers,
    calendarDate={"gte": start_date},
    dimension="ARY",
    qopts={"columns": [
        "ticker","datekey","calendarDate","revenue","eps","ebitda","netinc",
        "roe","evebitda","fcfps","pb","pe","marketcap","ebit","currentratio","capex","ncfo","equity","de"
    ]},
    paginate=True
)
fundamental = fundamental.rename(columns={"calendardate":"date"})
fundamental["date"]    = pd.to_datetime(fundamental["date"])
fundamental["year"] = fundamental["date"].dt.to_period("Y")

fundamental.to_csv(r'data\fundamental.csv')


In [37]:
max_datekey = fundamental['datekey'].max()
print(max_datekey)

2025-05-23 00:00:00


## 3. Descarga precios diarios desde 1998

In [20]:
ndl.export_table('SHARADAR/SEP',ticker= tickers, date = {"gte":start_date}, filename='prices.zip')

#### Debido al gran tamaño de _prices_ (1132 empresas, datos diarios, 27 años) es necesario descargar comprimible 

In [ ]:
# 1) Abre el ZIP y localiza el CSV de precios
zip_path = r'data\prices.zip'   # ajusta si tu ZIP se llama distinto o está en otra ruta
with zipfile.ZipFile(zip_path, 'r') as z:
    # busca el primer fichero que termine en .csv
    precios_csv = next(fname for fname in z.namelist() if fname.lower().endswith('.csv'))
    # carga el CSV parseando la columna 'date'
    prices = pd.read_csv(z.open(precios_csv), parse_dates=['date'])
    prices.to_csv(r'data\prices.csv.gz', index=False, compression='gzip') #Guasrdamos como versión comprimida de csv (7M rows)


## Crea variables momentum 6M y 12M lagged (Tras trabajar, poner en otro documento y arreglar extraccion, de csv en vez de zip)

In [ ]:
import pandas as pd
import zipfile

# 1) Abre el ZIP y localiza el CSV de precios
zip_path = r'..\data\prices.zip'   # ajusta si tu ZIP se llama distinto o está en otra ruta
with zipfile.ZipFile(zip_path, 'r') as z:
    # busca el primer fichero que termine en .csv
    precios_csv = next(fname for fname in z.namelist() if fname.lower().endswith('.csv'))
    # carga el CSV parseando la columna 'date'
    prices = pd.read_csv(z.open(precios_csv), parse_dates=['date'])

# 2) Normaliza nombres, comprueba que existan e índice ordenado
prices = prices.rename(columns=str.lower)   # 'Ticker'→'ticker', 'Date'→'date', etc.
required = ['ticker','date','closeadj']
assert all(col in prices.columns for col in required), "Falta alguna de: ticker, date, closeadj"
prices = prices.sort_values(['ticker','date']).reset_index(drop=True)

# 3) Extrae cierres ajustados al final de cada mes
monthly = (
    prices
    .groupby('ticker')
    .resample('ME', on='date')['closeadj']  # 'ME' = month end
    .last()                                 # último valor del mes
    .dropna()                               # elimina meses vacíos
    .reset_index()
)

# 4) Calcula retornos a 6 y 12 meses
for m in (6, 12):
    monthly[f'ret_{m}m'] = (
        monthly
        .groupby('ticker')['closeadj']
        .pct_change(periods=m)
    )

# 5) Filtra solo los finales de marzo (rebalanceo anual)
rebalance = monthly[monthly['date'].dt.month == 3].copy()

# 6) (Opcional) Si quisieras renombrar la fecha a 'rebalance_date'
rebalance = rebalance.rename(columns={'date':'rebalance_date'})

# 7) Guarda en CSV
rebalance.to_csv(r'..\data\rebalance_returns.csv', index=False)

# 8) Vista previa
print(rebalance.head())


   ticker rebalance_date  closeadj    ret_6m   ret_12m
4       A     2000-03-31    63.265       NaN       NaN
16      A     2001-03-31    18.694 -0.372073 -0.704513
28      A     2002-03-31    21.267  0.788195  0.137638
40      A     2003-03-31     7.999  0.006797 -0.623877
52      A     2004-03-31    19.241  0.430558  1.405426


## Comprobaciones para momentum max y min 

## Comprobaciones para Momementum, nan para primeras 6 y 11 filas de ret_6 y ret_11

In [ ]:
# Supongamos que tu DataFrame mensual se llama `monthly`
# y que ya tiene columnas ['ticker','date','closeadj','ret_6m','ret_11m',...]

def mostrar_retornos(ticker):
    # 1) Filtrar sólo ese ticker y ordenar por fecha
    df_t = (monthly
            .loc[monthly['ticker'] == ticker]
            .sort_values('date')
            .reset_index(drop=True))
    
    # 2) Mostrar las primeras 12 filas (un año)
    print(f"Retornos para {ticker}:\n")
    display(df_t[['date','closeadj','ret_6m','ret_12m']].head(12))

# Ejemplo: revisa el primer año de AAPL
mostrar_retornos('AAPL')


Retornos para AAPL:



,date,closeadj,ret_6m,ret_11m
0,1998-01-31,0.138,NaN,NaN
1,1998-02-28,0.178,NaN,NaN
2,1998-03-31,0.207,NaN,NaN
3,1998-04-30,0.206,NaN,NaN
4,1998-05-31,0.200,NaN,NaN
5,1998-06-30,0.216,NaN,NaN
6,1998-07-31,0.260,0.884058,NaN
7,1998-08-31,0.234,0.314607,NaN
8,1998-09-30,0.287,0.386473,NaN
9,1998-10-31,0.279,0.354369,NaN


## Crear varible técnicas

In [ ]:
import pandas as pd

# 1. Carga y ordenación
df = pd.read_csv(
    r"..\data\prices.csv.gz",
    compression="gzip",
    parse_dates=["date"]
)
df = df.sort_values(["ticker", "date"])
df["year"] = df["date"].dt.year

# 2. Último día de marzo de cada año y ticker
march = df[df["date"].dt.month == 3]
rebalance = (
    march
    .groupby(["ticker", "year"], as_index=False)
    .agg(rebalance_date=("date", "max"))
)

# 3. Cálculo de features por ticker+year usando precios ajustados (closeadj)
records = []
for _, row in rebalance.iterrows():
    tkr = row["ticker"]
    d0  = row["rebalance_date"]
    # Ventana de 1 y 2 años antes del rebalance
    win1 = df[
        (df["ticker"] == tkr) &
        (df["date"] >  d0 - pd.Timedelta(days=365)) &
        (df["date"] <= d0)
    ]
    win2 = df[
        (df["ticker"] == tkr) &
        (df["date"] >  d0 - pd.Timedelta(days=730)) &
        (df["date"] <= d0)
    ]
    if win1.empty:
        continue

    records.append({
        "ticker":         tkr,
        "year":           row["year"],
        "rebalance_date": d0,
        # Último cierre ajustado antes del rebalance
        "close_reb_adj":  win1["closeadj"].iloc[-1],
        "vol_1y":         win1["volume"].sum(),
        "max_1y":         win1["closeadj"].max(),
        "min_1y":         win1["closeadj"].min(),
        "sma_1y":         win1["closeadj"].mean(),
        "sma_2y":         win2["closeadj"].mean() if not win2.empty else None,
        "volatility_1y":  win1["closeadj"].pct_change().std() * (252**0.5)
    })

tecnico = pd.DataFrame(records)

# 4. Guardar en CSV
tecnico.to_csv(r"..\data\tecnico.csv", index=False)


In [2]:
tecnico.head()

,ticker,year,rebalance_date,close_reb_adj,vol_1y,max_1y,min_1y,sma_1y,sma_2y,volatility_1y
0,A,2000,2000-03-31,63.265,220099000.0,96.723,24.485,48.119387,48.119387,1.188489
1,A,2001,2001-03-30,18.694,847576400.0,74.215,18.250,35.402956,38.759834,0.951615
2,A,2002,2002-03-28,21.267,571608700.0,24.576,11.473,17.982242,26.952406,0.582828
3,A,2003,2003-03-31,7.999,677176600.0,22.222,6.600,11.631502,14.760365,0.656080
4,A,2004,2004-03-31,19.241,653408900.0,22.806,8.352,15.060278,13.314752,0.349033
